In [ ]:
# https://gist.github.com/DrSleep/4bce37254c5900545e6b65f6a0858b9c
import tensorflow as tf
from PIL import Image

In [ ]:
from tqdm import tqdm

In [ ]:
import numpy as np

In [ ]:
arr = Image.open('rwth_kitti_semantics_dataset/labels/06_000800.png')

In [ ]:
arr

In [ ]:
np.array(arr).shape

In [ ]:
arr = np.array(arr)
dt = np.dtype(['a', arr.dtype), ('b', arr.dtype), ('c', arr.dtype)])
y = arr.view(dtype=dt).squeeze()

In [ ]:
np.unique(y)

In [ ]:
## convert labels from RGB

In [ ]:
## adapted from https://github.com/martinkersner/train-DeepLab/blob/master/utils.py
palette = {(0,   0,   0) : 0 ,
         (0,  0, 255) : 1 ,
         (255,  0,  0) : 2,
           (255,255,  0) : 3,
           (  0,255,  0) : 4,
           (255,  0,255) : 5,
           (  0,255,255) : 6,
           (255,  0,153) : 7,
           (153,  0,255) : 8,
           (  0,153,255) : 9,
           (153,255,  0) : 10,
           (255,153,  0) : 11,
           (  0,255,153) : 12,
           (  0,153,153) : 13
          }

In [ ]:
def convert_from_color_segmentation(arr_3d):
    arr_2d = np.zeros((arr_3d.shape[0], arr_3d.shape[1]), dtype=np.uint8)

    for c, i in palette.items():
        m = np.all(arr_3d == np.array(c).reshape(1, 1, 3), axis=2)
        arr_2d[m] = i

    return arr_2d

In [ ]:
Image.fromarray(convert_from_color_segmentation(arr))

In [ ]:
import os, shutil

In [ ]:
new_label_dir = 'rwth_kitti_semantics_dataset/labels_tf/'

In [ ]:
label_dir = 'rwth_kitti_semantics_dataset/labels/'
label_files = os.listdir(label_dir)

In [ ]:
for l_f in tqdm(label_files):
    arr = np.array(Image.open(label_dir + l_f))
    arr_2d = convert_from_color_segmentation(arr)
    Image.fromarray(arr_2d).save(new_label_dir + l_f)

In [ ]:
## check
np.array(Image.open('rwth_kitti_semantics_dataset/labels_tf/00_000200.png')).shape

In [ ]:
## fix training/test sets

In [ ]:
with open('rwth_kitti_semantics_dataset/splits/train.txt', 'rb') as f:
    trn_lst = list(f.readlines())

In [ ]:
with open('rwth_kitti_semantics_dataset/splits/test.txt', 'rb') as f:
    tst_lst = list(f.readlines())

In [ ]:
## we want /train/file1.png /labels_tf/file1.png
with open('rwth_kitti_semantics_dataset/splits/train_tf.txt', 'w+') as f:
    for el in trn_lst:
        el = el.decode('utf-8')[:-5]
        f.write('/images/{}.png /labels_tf/{}.png\n'.format(el, el))

In [ ]:
## we want /train/file1.png /labels_tf/file1.png
with open('rwth_kitti_semantics_dataset/splits/test_tf.txt', 'w+') as f:
    for el in tst_lst:
        el = el.decode('utf-8')[:-5]
        f.write('/images/{}.png /labels_tf/{}.png\n'.format(el, el))

In [ ]:
#Download dataset from https://omnomnom.vision.rwth-aachen.de/data/rwth_kitti_semantics_dataset.zip
#Convert RGB label images using masks_kitti.ipynb
#Download deeplab_resnet.ckpt
#Change fine_tune.py configuration (example is given)
#Change https://github.com/DrSleep/tensorflow-deeplab-resnet/blob/master/deeplab_resnet/image_reader.py#L115 to decode_png
#Run fine_tune.py